### install required libraries 

In [1]:
!pip install -q  google-generativeai python-dotenv

### Load Environment variables 

In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()
gemini_key = os.environ.get('GEMINI_API_KEY')
openweather_key = os.environ.get('OPENWEATHERMAP_API_KEY')
if gemini_key and openweather_key:
    print('API keys fetched successfully')

API keys fetched successfully


### import necessary modules 

In [3]:
import google.generativeai as genai
import re 
import os 
import requests
from datetime import datetime,timedelta

/home/mukesh/Github-my repos/Generative AI projects/TravelGPT/travelGPT/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Creating the Agent Class

In [4]:


class ChatHistoryManager:
    def __init__(self, filename="travel_agent_chat_history.txt", max_file_size_mb=5):
        self.history = []
        self.filename = filename
        self.max_file_size_mb = max_file_size_mb

    def add_message(self, role, text):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.history.append(
            {'role': role, 'text': text, 'timestamp': timestamp})

    def save_to_file(self):
        self._rotate_file_if_needed()
        with open(self.filename, "a", encoding="utf-8") as file:
            for message in self.history:
                file.write(
                    f"{message['timestamp']} {message['role']}: {message['text']}\n")
        self.history.clear()

    def display(self):
        for message in self.history:
            print(
                f"{message['timestamp']} {message['role']}: {message['text']}")

    def _rotate_file_if_needed(self):
        if not os.path.exists(self.filename):
            with open(self.filename, "a", encoding="utf-8") as file:
                pass

        if os.path.getsize(self.filename) > self.max_file_size_mb * 1024 * 1024:
            os.rename(self.filename, self.filename + ".backup")


class TravelAgent:
    def __init__(self, system_prompt, initial_budget=1000):
     

        # Configure Generative AI
        genai.configure(api_key=gemini_key)

        self.system_prompt = system_prompt
        self.budget = initial_budget
        self.history_manager = ChatHistoryManager()
        self.history_manager.add_message("system", system_prompt)
        self.history_manager.add_message("system", f"The current budget is ${self.budget}.")
        
        self.generation_config = {
            "temperature": 0.1,
            "top_p": 1,
            "top_k": 1,
            "max_output_tokens": 2048,
        }

        self.safety_settings = {
            "HARM_CATEGORY_HARASSMENT": "BLOCK_NONE",
            "HARM_CATEGORY_HATE_SPEECH": "BLOCK_NONE",
            "HARM_CATEGORY_SEXUALLY_EXPLICIT": "BLOCK_NONE",
            "HARM_CATEGORY_DANGEROUS_CONTENT": "BLOCK_NONE",
        }

        self.model = genai.GenerativeModel(
            'gemini-1.5-flash', generation_config=self.generation_config, safety_settings=self.safety_settings
        )
        self.chat = self.model.start_chat(history=[])

    def __call__(self, user_input):
        if user_input.lower() == "history":
            self.history_manager.display()
            return

        if user_input.lower() == "restart":
            self.history_manager.save_to_file()
            os.system('cls' if os.name == 'nt' else 'clear')
            self.history_manager.add_message("system", "--- New Session ---")
            self.chat = self.model.start_chat(history=[])
            return "Session restarted."

        if user_input.lower() == "exit":
            self.history_manager.save_to_file()
            return "Session ended. Chat history saved."

        try:
            # Append user input to history and send to model
            self.history_manager.add_message("user", user_input)
            response = self.chat.send_message(user_input, stream=True)
            
            response_text = ""
            for chunk in response:
                if chunk.text.endswith("."):
                    response_text += chunk.text
                else:
                    response_text += re.sub(r'\s*$', '.', chunk.text)
                print(chunk.text, end="", flush=True)  # Stream response to user

            # Save response to history
            self.history_manager.add_message("assistant", response_text)
            return response_text

        except Exception as e:
            return f"An error occurred: {e}"

    def set_budget(self, amount):
        self.budget = amount
        self.history_manager.add_message("system", f"The current budget is now ${self.budget}.")

    def get_budget(self):
        return self.budget


## Tools for the Agent 

### Currency convertor 


In [5]:
def convert_currency(amount,from_currency,to_currency):
    import requests
    url = "https://api.exchangerate-api.com/v4/latest/" + from_currency
    response = requests.get(url)
    data = response.json()
    rate = data['rates'][to_currency]
    converted_amount = amount * rate
    return round(converted_amount,2)


In [6]:
convert_currency(100,'USD','INR')

8497.0

### Weather Tool

In [7]:
def get_weather(location:str,days_ahead:int =0)->str:
    import datetime
    api_key = os.environ.get("OPENWEATHERMAP_API_KEY") 
    future_date = datetime.date.today() + datetime.timedelta(days=days_ahead)
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={location}&appid={api_key}&units=metric&dt={int(datetime.datetime(future_date.year,future_date.month,future_date.day,12,0,0).timestamp())}"
    response = requests.get(url)
    data = response.json()
    if response.status_code == 200:
        try:
            temperature = data['list'][0]['main']['temp']
            description = data['list'][0]['weather'][0]['description']
            country = data['city']['country'] #Added this line to extract country
            return f"The weather in {location}, {country} on {future_date} will be {description} with a temperature of {temperature}°C."
        except (KeyError, IndexError):
            return f"Error getting weather data for {location} on {future_date}.  Data not found in response."

    else:
        return f"Error getting weather data for {location} on {future_date}. Status code: {response.status_code}"


In [8]:
get_weather('Dubai',3)

'The weather in Dubai, AE on 2024-12-21 will be broken clouds with a temperature of 21.48°C.'

### Translation tool

In [9]:
from deep_translator import GoogleTranslator

def translate_text(text, target_language):
    """
    Translate text into the specified target language using deep-translator.

    Parameters:
        text (str): The text to translate.
        target_language (str): The target language code (e.g., 'en' for English, 'fr' for French).

    Returns:
        str: The translated text.
    """
    try:
        # Initialize the translator
        translator = GoogleTranslator(source='auto', target=target_language)
        
        # Perform translation
        translated_text = translator.translate(text)
        
        return translated_text
    except Exception as e:
        return f"An error occurred: {e}"

In [10]:
# Example usage:
original_text = "Hola, ¿cómo estás?"
target_language = "en"  # Translate to English
translated_text = translate_text(original_text, target_language)
print("Translated text:", translated_text)

Translated text: Hello how are you?


### Prompt 

In [11]:
travel_agent_prompt=""" You are a Travel Planning Assistant. Your role is to help users plan their trips by providing information about destinations, currency conversion, weather forecasts, and language translation.

You run in a loop of Thought, Action, PAUSE, Observation.
Use Thought to describe your reasoning about the user's request.
Use Action to perform one of the available actions, then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

1. convert_currency:
             e.g., convert_currency: 100, USD, EUR
             Converts the given amount from one currency to another.

2. get_weather:
             e.g. get_weather: Paris, 3
             Gets the weather forecast for the specified city and number of days ahead (e.g. for today).

3. translate_text:
             e.g., translate_text: Hello, how are you?, fr
             Translates the given text to the specified target language (use language codes like 'fr' for French).

Always consider the user's budget when making suggestions. The current budget will be provided in the system messages.

Example session:

User: I'm planning a trip to Paris next week. What should I know?

Thought: I should check the weather in Paris for next week and provide some basic French phrases.
Action: get_weather: Paris, 7

PAUSE

Observation: Partly cloudy, 22°C

Thought: Now that I have the weather information, I should provide a simple French phrase and its translation.
Action: translate_text: Hello, how are you?, fr

PAUSE

Observation: Bonjour, comment allez-vous?

Answer: Great! For your trip to Paris next week, here's what you should know:

1. Weather: It's expected to be partly cloudy with a temperature of 22°C (72°F). Pack accordingly with light layers and maybe a light jacket for evenings.

2. Language: While many people in Paris speak some English, it's always appreciated if you try some basic French. Here's a useful phrase: "Hello, how are you?" in French is "Bonjour, comment allez-vous?"

Remember to consider the budget for your trip and let me know if you need any currency conversion or have any other questions about your Paris adventure!

"""

## Buidling  the AI Travel Agent 



### Directory of available actions 


In [12]:
known_actions = {
    "convert_currency": convert_currency,
    "get_weather": get_weather,
    "translate_text": translate_text
}


### format markdown function

In [13]:
def format_markdown(markdown_text):
        """Converts markdown text to a well-formatted preview."""
        try:
            from IPython.display import Markdown, display
            display(Markdown(markdown_text))
        except ImportError:
            print("IPython.display not found. Please install it using 'pip install ipython'")
            print(markdown_text)

### AI Travel Agent 

In [14]:
def plan_trip(question,max_turns=1):
    agent =TravelAgent(travel_agent_prompt)

    budget_match=re.search(r'\$(\d+)',question)
    if budget_match:
        budget=int(budget_match.group(1))
        agent.set_budget(budget)

    next_prompt=question
    
    action_re=re.compile(r'^Action:(\w+):(.+)$')

    for i in range(max_turns):
        result=agent(next_prompt)
        format_markdown(result)

        actions=[action_re.match(a) for a in result.split('\n') if action_re.match(a)]

        if actions:
            action,action_input=actions[0].groups()

            if action not in known_actions:
                print(f"Unknown action :{action}:{action_input}")
                continue

            print(f'-- running {action}:{action_input}')

            try:
                if action =='convert_currency':
                    amount,from_currency,to_currency=action_input.split(',')
                    observation=known_actions[action](float(amount),from_currency.strip(),to_currency.strip())
                elif action == 'get_weather':
                    location,*days=action_input.split(',')
                    days_ahead=int(days[0]) if days else 0
                    observation=known_actions[action](location.strip(),days_ahead)
                elif action =='translate_text':
                    if ',' in action_input:
                        text,target_lang=action_input.rsplit(',',1)
                    else:
                        text ,target_lang= action_input.rsplit(None,1)

                    observation = known_actions[action](text.strip(),target_lang.strip())

                print(f'Observation : {observation}')
                agent.add_to_memory(action,observation)
                next_prompt= f'Observation:{observation}'
            except  Exception as e:
                print(f"Error executing action :{e}")
                next_prompt=f'Error : Unable to execute action {action}.please try again'
        else:
            return
                         
                                                      

### Experiment


In [16]:
question = "i am planing a 3 days  trip to vizag in the next week . my budget is 2000 INR .what  should i prepare ?"
result=plan_trip(question)

A 3-day trip to Visakhapatnam (Vizag) on a ₹2000 budget will require careful planning.  This budget is extremely tight, especially if you're travelling from outside Vizag.  It's likely you'll need to make compromises on comfort and potentially skip some attractions.  Here's a possible plan, focusing on maximizing your experience within your constraints:

**Before you go:**

* **Transportation:**  The biggest chunk of your budget will likely be transportation. If you're travelling from another city, consider the cheapest option – a state-run bus.  Book in advance for better prices. Within Vizag, rely heavily on auto-rickshaws (negotiate fares beforehand) or city buses (very cheap).
* **Accommodation:**  Forget hotels.  Your budget necessitates staying in a hostel or a very basic guesthouse. Look for options well in advance on platforms like MakeMyTrip or Goibibo, filtering by price.  Consider sharing a room to reduce costs.  Alternatively, if you know someone in Vizag, staying with them

A. 3-day trip to Visakhapatnam (Vizag) on a. ₹2000 budget will require careful planning.  This budget is extremely. tight, especially if you're travelling from outside Vizag.  It's likely you'll need to make compromises on comfort and potentially skip some attractions..  Here's a possible plan, focusing on maximizing your experience within your constraints:

**Before you go:**

* **Transportation:**  The biggest. chunk of your budget will likely be transportation. If you're travelling from another city, consider the cheapest option – a state-run bus.  Book in advance for better prices. Within Vizag, rely heavily on auto-ricksha.ws (negotiate fares beforehand) or city buses (very cheap).
* **Accommodation:**  Forget hotels.  Your budget necessitates staying in a hostel or a very basic guesthouse. Look for options well in advance on platforms like MakeMy.Trip or Goibibo, filtering by price.  Consider sharing a room to reduce costs.  Alternatively, if you know someone in Vizag, staying with them would be ideal.
* **Food:**  Street food is your friend!  Avoid restaurants unless it's a very special occasion.  Pack some. snacks and water bottles to avoid frequent food purchases.  Local eateries will offer much cheaper meals than tourist-oriented restaurants.
* **Packing:**  Pack light! This saves on baggage fees if travelling by bus.  Essentials include comfortable walking shoes, light clothing suitable for warm weather, sunscreen, a hat, insect repellent., and any necessary medications.
* **Itinerary:**  Plan your sightseeing carefully. Prioritize free or low-cost activities.

**Possible 3-Day Itinerary (Budget-Friendly):**

**Day 1:**

* **Morning:** Arrive in Vizag, check into your accommodation.
* **.Afternoon:** Visit RK Beach (free). Enjoy the sea breeze, maybe a small street food snack.
* **Evening:** Explore the beach further, watch the sunset.

**Day 2:**

* **Morning:** Visit Kailasagiri Hill Park (entry fee applicable, but relatively inexpensive). Enjoy the views..  Consider walking up instead of taking the ropeway to save money.
* **Afternoon:**  Explore the local markets for souvenirs (be prepared to bargain hard).
* **Evening:**  Enjoy street food for dinner.

**Day 3:**

* **Morning:** Visit the VUDA Park (free. or minimal entry fee).
* **Afternoon:**  Depending on time and energy, you could visit a temple (many are free to enter).
* **Evening:** Depart from Vizag.

**Important Considerations:**

* **₹2000 is extremely tight:** This budget might only cover your travel and. basic accommodation. Food and entry fees to attractions will eat into this significantly.  You might need to adjust your expectations or find ways to supplement your budget.
* **Negotiate prices:**  Always negotiate with auto-rickshaw drivers and street vendors.
* **Travel during off-season:** If possible, avoid. peak tourist season for better deals on accommodation.
* **Prioritize:** Choose the attractions that are most important to you and focus on those.

**To make this work, you'll likely need to:**

* **Travel during the off-season:** This will significantly reduce accommodation costs.
* **Stay. in a dorm or shared room:** This is the only way to keep accommodation costs low.
* **Eat mostly street food:** This is the cheapest and most authentic way to experience Vizag's culinary scene.
* **Walk or use public transport:** This will save you a lot of money on transportation...Realistically, ₹2000 might be insufficient for a comfortable 3-day trip to Vizag unless you have significant external support (free accommodation, etc.).  Consider revising your budget or trip duration for a more enjoyable experience...